In [1]:
import os
os.chdir('/Users/arshiayousefi/Desktop/ML/armanemo')
os.getcwd()

'/Users/arshiayousefi/Desktop/ML/armanemo'

In [2]:
import pandas as pd
import torch
import time
from emotion_detection.models.models_DL import TextClassificationRNN, train, evaluate
from emotion_detection.preprocess.process import process_length_words
from emotion_detection.loader.data_loader import split_data
from emotion_detection.loader.data_loader_deepmodel import create_data_loader, tokens_ids

In [7]:
print('hi')
data = pd.read_csv("/Users/arshiayousefi/Desktop/ML/armanemo/data/train_process2.csv")
data = process_length_words(data)
label_list = list(sorted(data['label'].unique()))
train_data, valid_data, test_data = split_data(data, 'label')

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {v: k for k, v in label2id.items()}

device = "cpu"
embedd_dim = 512
hidden_dim = 256
output_dim = len(label2id)
pad_idx = 0
n_epochs = 10
# Hyperparameters
EPOCHS = 10  # epoch
lr = 1e-1  # learning rate
batch_size = 128
MAX_VOCAB = 50_000

# Assuming 'class_counts' is a list or array with the number of samples for each class
class_counts = train_data.label.value_counts().to_dict()

total_samples = sum(class_counts.values())
class_weights = {class_label: total_samples / class_count for class_label, class_count in enumerate(class_counts.values())}

train_data, valid_data, test_data, token2id = tokens_ids(train_data, valid_data, test_data, MAX_VOCAB)
train_loader, valid_loader, test_loader = create_data_loader(train_data, valid_data, test_data, label2id, batch_size, device)
vocab_size = len(token2id) + 1

model = TextClassificationRNN(vocab_size, embedd_dim, hidden_dim, output_dim, pad_idx)
# model = model.to(device)

criterion = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(list(class_weights.values())).to(device))
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_loader)
    accu_train, loss_train = evaluate(train_loader)
    accu_val, loss_val = evaluate(valid_loader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        " | train accuracy {:8.3f} | loss train {:8.3f} | valid accuracy {:8.3f} | loss validation {:8.3f}".format(
            epoch, time.time() - epoch_start_time, accu_train, loss_train, accu_val, loss_val
        )
    )
    print("-" * 59)

hi
maximum vocab numbers:  3714


AssertionError: Torch not compiled with CUDA enabled

In [1]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.2
Is CUDA enabled? False
